In [2]:
import json
import glob
import pandas as pd
from datetime import datetime
from collections import defaultdict

In [7]:
json_files = glob.glob("compoundV2_transactions_ethereum_chunk_*.json")

transactions = {
    "deposits": [],
    "borrows": [],
    "repays": [],
    "withdraws": [],
    "liquidates": []
}

for file in json_files:
    with open(file, 'r') as f:
        data = json.load(f)
        for key in transactions:
            if key in data:
                transactions[key].extend(data[key])

# Check how many transactions per type
{key: len(transactions[key]) for key in transactions}


{'deposits': 29305, 'borrows': 0, 'repays': 0, 'withdraws': 0, 'liquidates': 0}

In [4]:
# Show top-level keys in one file
with open("compoundV2_transactions_ethereum_chunk_89.json") as f:
    data = json.load(f)
    print(data.keys())


dict_keys(['deposits', 'withdraws', 'borrows', 'repays', 'liquidates'])


In [13]:
wallet_features = defaultdict(lambda: {
    "total_deposit_usd": 0,
    "tx_timestamps": [],
    "tx_count": 0
})

def safe_float(x):
    try:
        return float(x)
    except:
        return 0.0

for tx in transactions["deposits"]:
    wallet = tx["account"]["id"]
    amount_usd = safe_float(tx.get("amountUSD", 0))
    timestamp = int(tx.get("timestamp", 0))
    
    wallet_features[wallet]["total_deposit_usd"] += amount_usd
    wallet_features[wallet]["tx_timestamps"].append(timestamp)
    wallet_features[wallet]["tx_count"] += 1



In [14]:
rows = []
for wallet, feats in wallet_features.items():
    timestamps = feats["tx_timestamps"]
    active_days = len(set(datetime.utcfromtimestamp(ts).date() for ts in timestamps))
    span_days = (max(timestamps) - min(timestamps)) / 86400 if timestamps else 0

    rows.append({
        "wallet": wallet,
        "total_deposit_usd": feats["total_deposit_usd"],
        "active_days": active_days,
        "span_days": span_days,
        "tx_count": feats["tx_count"]
    })

wallet_df = pd.DataFrame(rows)
wallet_df.head()


,wallet,total_deposit_usd,active_days,span_days,tx_count
0,0x013022c7ca735ef6ba55cd3eaccb4ebb3be484e9,5.441944e+05,48,134.928056,52
1,0x8888882f8f843896699869179fb6e4f7e3b58888,4.620348e+08,153,786.580613,2139
2,0x9cc4e339a7636c5dcc7b039aa7ca29dd920bcf7b,7.247031e+04,5,15.996944,8
3,0xf0163f66ec80dda288e753e0a62c8eb71cd38684,1.836147e+07,251,1021.284699,695
4,0x380bae3560cc2469bbae7cf5fa3810668433756b,1.000000e+02,1,0.000000,1


In [ ]:
from sklearn.preprocessing import MinMaxScaler

def compute_score(row):
    score = 0
    score += row["total_deposit_usd"] * 0.01
    score += row["active_days"] * 2
    score += row["span_days"] * 1
    score += row["tx_count"] * 0.5
    return score

wallet_df["raw_score"] = wallet_df.apply(compute_score, axis=1)

scaler = MinMaxScaler((0, 100))
wallet_df["score"] = scaler.fit_transform(wallet_df[["raw_score"]])
wallet_df = wallet_df.sort_values("score", ascending=False)
wallet_df[["wallet", "score"]].head(10)

,wallet,score
3782,0x342491c093a640c7c2347c4ffa7d8b9cbc84d1eb,100.000000
2482,0x352423e2fa5d5c99343d371c9e3bc56c87723cc7,87.586303
1,0x8888882f8f843896699869179fb6e4f7e3b58888,85.799356
4360,0x10d88638be3c26f3a47d861b8b5641508501035d,84.950596
8,0xb99cc7e10fe0acc68c50c7829f473d81e23249cc,60.513830
2119,0xed0c6079229e2d407672a117c22b62064f4a4312,57.745287
4463,0xdbebc7fc0d343cc31efee75c5a3f8982a39c3268,43.839510
3400,0x1676055fe954ee6fc388f9096210e5ebe0a9070c,38.983237
4330,0x466e89db9e88a69367e4e9aae5b6ff508f8e2cfe,35.835806
2676,0xabde2f02fe84e083e1920471b54c3612456365ef,34.128217


In [19]:
wallet_df[["wallet", "score"]].head(1000).to_csv("top_wallet_scores.csv", index=False)


In [20]:
top5 = wallet_df.head(5)
bottom5 = wallet_df.tail(5)

pd.concat([top5, bottom5])[[
    "wallet", "score", "total_deposit_usd", "active_days", "span_days"
]]


,wallet,score,total_deposit_usd,active_days,span_days
3782,0x342491c093a640c7c2347c4ffa7d8b9cbc84d1eb,100.000000,5.387408e+08,15,104.725949
2482,0x352423e2fa5d5c99343d371c9e3bc56c87723cc7,87.586303,4.718585e+08,4,187.105694
1,0x8888882f8f843896699869179fb6e4f7e3b58888,85.799356,4.620348e+08,153,786.580613
4360,0x10d88638be3c26f3a47d861b8b5641508501035d,84.950596,4.576557e+08,47,95.499144
8,0xb99cc7e10fe0acc68c50c7829f473d81e23249cc,60.513830,3.258240e+08,293,1021.004120
410,0xbef0481105bb3e79056219f9e870814e98a774fe,0.000000,0.000000e+00,1,0.000000
97,0x93fa1fc2886b0eed3da29b5cc079c3a887f4a9e9,0.000000,0.000000e+00,1,0.000000
96,0xc002a6a475cdd97cda49829b7d2aa9189c49867d,0.000000,0.000000e+00,1,0.000000
749,0xf1c70ede18cfed1d70253bdbabaeb845dae714bc,0.000000,0.000000e+00,1,0.000000
7139,0xd745c719b5f0a58d561953832f279f869a43f2e9,0.000000,0.000000e+00,1,0.000000
